In [1]:
%matplotlib inline
import pystan
import numpy as np
import matplotlib.pyplot as plt

import sys
import math
import random
import numpy as np
from scipy.io import loadmat

import plotly.offline as pyo
import plotly.graph_objs as go

pyo.init_notebook_mode(connected=True)

In [2]:
# List of timestep sizes
ALLDELTA = range(100)
# Fontsize parameter for plotting
FS = 28

# Dataset
NETDIR = 'primaryschool.csv'
# ID file
IDDIR = 'id_class_gender'

# Import data
ndat = open(NETDIR, 'r').readlines()
ndat = np.asarray([np.asarray([elem for elem in line.strip().split('\t')[:-2]]+line.strip().split('\t')[-2:]) for line in ndat])

# Get all time points
all_t = np.asarray([int(elem) for elem in ndat[:,0]])
# Zero time points
all_t -= min(all_t)
min_t = min(all_t)
max_t = max(all_t)
assert(min_t==0)

# Get all node ID's
all_n = list(set([elem for elem in ndat[:,1]]+[elem for elem in ndat[:,2]]))
# Get all class ID's
all_d = list(set([elem for elem in ndat[:,3]]+[elem for elem in ndat[:,4]]))
nnodes = len(all_n)
ndept = len(all_d)

all_nodes = []
for delta in ALLDELTA:
    
    curr_ndat = np.copy(ndat)
    curr_max_t = max_t
    curr_all_t = np.copy(all_t)
    
    # Coarse-grain time resolution
    # Units of time steps are seconds
    # Time steps are given in 20 second chunks
    curr_all_t //= 60*(delta+1)
    curr_max_t //= 60*(delta+1)
    max_min_diff = []

    all_A_d = []
    
    for ts in range(curr_max_t):

        curr_filt_ts = curr_all_t==ts

        ccurr_ndat = curr_ndat[curr_filt_ts]
        if len(ccurr_ndat.shape)<2:
            ccurr_ndat = np.asarray([ccurr_ndat])
        assert(len(ccurr_ndat.shape)==2)

        # Compute adjacency matrix of interactions that take place during time ts
        A_d = np.zeros((ndept+1,ndept+1), dtype=float)
        
        for row in ccurr_ndat:
            
            nrind1, nrind2 = all_n.index(row[1]), all_n.index(row[2])
            drind1, drind2 = all_d.index(row[3]), all_d.index(row[4])
            
            A_d[drind1][drind2] += 1.
            
        all_A_d.append(A_d.flatten())
    all_A_d = np.asarray(all_A_d).reshape((len(all_A_d),12,12)).astype(int)
    all_nodes.append(all_A_d)

In [3]:
function_code = """
functions {
    
    matrix left_rotation(matrix A, real angle, int n, int p, int i, int j) {
        matrix[n, p] RA;
        RA = A;
        
        RA[i,] = cos(angle)*A[i,] - sin(angle)*A[j,];
        RA[j,] = sin(angle)*A[i,] + cos(angle)*A[j,];
        
        return RA;
    }
    
    matrix right_rotation(matrix A, real angle, int n, int i, int j) {
        matrix[n, n] AR;
        AR = A;
        
        AR[,i] = cos(angle)*A[,i] + sin(angle)*A[,j];
        AR[,j] = -sin(angle)*A[,i] + cos(angle)*A[,j];
        
        return AR;
    }
    
    matrix d_rotation_matrix(real angle, int n, int i, int j) {
        matrix[n, n] dR = diag_matrix(rep_vector(0, n));
        
        dR[i, i] = -sin(angle);
        dR[i, j] = -cos(angle);
        dR[j, i] = cos(angle);
        dR[j, j] = -sin(angle);
        
        return dR;
    }
    
    matrix[] generate_forward_pgivens(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] G;
        matrix[n, n] partial_givens[d+1];
        matrix[n, n] R;
        int pp;
        if(p == n) pp = p - 1;
        else pp = p;
        
        G = diag_matrix(rep_vector(1, n));
        idx = 1;
        partial_givens[1] = G;
        for(i in 1:pp){
            for(j in i+1:n){
                G = right_rotation(G, angles[idx], n, i, j);
                partial_givens[idx + 1] = G;
                idx = idx + 1;
            }
        }
        
        return partial_givens;      
    }
    
    matrix[] generate_reverse_pgivens(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] G_eye;
        matrix[n, p] G;
        matrix[n, p] partial_givens[d+1];
        matrix[n, n] R;
        int pp;
        if(p == n) pp = p - 1;
        else pp = p;

        G_eye = diag_matrix(rep_vector(1, n));
        G = G_eye[,1:p];
        
        partial_givens[d+1] = G;
        idx = d;
        for(i in 1:pp){
            int i_st = pp - i + 1;
            for(j in i_st+1:n){
                G = left_rotation(G, angles[idx], n, p, i_st, n - j + i_st + 1);
                partial_givens[idx] = G;
                idx = idx - 1;
            }
        }
        
        return partial_givens;      
    }
    
    matrix[] generate_givens_jacobians(matrix[] partial_givens_forward, matrix[] partial_givens_reverse, vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        matrix[n,p] derivative_list[d];
        matrix[n,d] givens_jacobian[p];
        int idx = 1;
        int pp;
        if(p == n) pp = p - 1;
        else pp = p;
        
        for(i in 1:p){
            for(j in i+1:n){
                matrix[n,n] dR = d_rotation_matrix(angles[idx], n, i, j);
                matrix[n,n] a = partial_givens_forward[idx];
                matrix[n,p] b = partial_givens_reverse[idx + 1];
                
                derivative_list[idx] = a * dR * b;
                idx = idx + 1;
            }
        }
        
        for(i in 1:pp) {
            for(j in 1:d) {
                vector[n] t = derivative_list[j][,i];
                matrix[n,d] z = givens_jacobian[i];
                z[,j] = t;
                givens_jacobian[i] = z;
            }
        }
        
        return givens_jacobian;
        
    }
    
    matrix area_form_lp(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] givens;
        matrix[n, n] partial_givens_forward[d+1];
        matrix[n, p] partial_givens_reverse[d+1];
        matrix[n, d] givens_jacobians[p];
        matrix[d, d] area_mat;
        int pp;
        if(p == n) pp = p - 1;
        else pp = p;
        

        partial_givens_forward = generate_forward_pgivens(angles, n, p);
        partial_givens_reverse = generate_reverse_pgivens(angles, n, p);
        givens = partial_givens_forward[d+1];
        
        givens_jacobians = generate_givens_jacobians(partial_givens_forward, partial_givens_reverse, angles, n, p);
        
        idx = 1;
        for(i in 1:pp){
            matrix[d, n-i] one_forms;
            one_forms = (givens'[i+1:n,] * givens_jacobians[i])';
            for(j in 1:n-i) {
                area_mat[,idx] = one_forms[,j]; 
                idx = idx + 1;
            }
        }
        
        target += log_determinant(area_mat); 
        return givens[,1:p];
    }
    
    matrix area_form(vector angles, int n, int p) {
        int d = n*p - p*(p+1)/2;
        int idx;
        matrix[n, n] givens;
        matrix[n, n] partial_givens_forward[d+1];
        matrix[n, p] partial_givens_reverse[d+1];
        matrix[n, d] givens_jacobians[p];
        matrix[d, d] area_mat;
        int pp;
        if(p == n) pp = p - 1;
        else pp = p;
        

        partial_givens_forward = generate_forward_pgivens(angles, n, p);
        partial_givens_reverse = generate_reverse_pgivens(angles, n, p);
        givens = partial_givens_forward[d+1];
        
        givens_jacobians = generate_givens_jacobians(partial_givens_forward, partial_givens_reverse, angles, n, p);
        
        idx = 1;
        for(i in 1:pp){
            matrix[d, n-i] one_forms;
            one_forms = (givens'[i+1:n,] * givens_jacobians[i])';
            for(j in 1:n-i) {
                area_mat[,idx] = one_forms[,j]; 
                idx = idx + 1;
            }
        }
    
        return givens[,1:p];
    }
}

data {
    int n;
    int p;
    int d;
    int N; // number of data points
    
    int<lower=1> K; // num categories
    int<lower=0> interactions[N,n,n];
    vector<lower=0>[K] beta;   // transition prior
}

transformed data {
}

parameters {
    simplex[K] phi[K]; //transit probs
    vector<lower = -pi()/2, upper = pi()/2>[d] theta[K]; // emission parameters
    vector[p] Lambda[K];
}

transformed parameters{
    matrix[n, n] Ws[K];
    
    for (k in 1:K) {
        matrix[n, p] Wt;
        matrix[p, p] diag_l;
        diag_l = diag_matrix(Lambda[k]);
        Wt = area_form_lp(theta[k], n, p);
        Ws[k] = Wt * diag_l * Wt';
    }
}

model {
    real acc[K];
    real gamma[N, K];
    
    for (k in 1:K)
        phi[k] ~ dirichlet(beta);
    
    // compute jacobian correction for area representation
    // and combined matrix
    for (k in 1:K) {
        gamma[1,k] = 0;
        for(x in 1:n) {
            for(y in x:n) {
                gamma[1,k] = gamma[1,k] + poisson_log(interactions[1,x,y], exp(Ws[k][x,y]));
            }
        }
    }
    
    // forwards algorithm to compute likelihood
    for (t in 2:N) {
        for (k in 1:K) {
            for (j in 1:K) {
                acc[j] = gamma[t-1,j] + log(phi[j,k]);
                for(x in 1:n) {
                    for(y in x:n) {
                        acc[j] = acc[j] + poisson_log(interactions[t,x,y], exp(Ws[k][x,y]));
                    }
                }
            }
            gamma[t,k] = log_sum_exp(acc);
        }
    }
    
    // accumulate likelihood
    target += log_sum_exp(gamma[N]);
}

generated quantities {
    int<lower=1, upper=K> y_star[N];
    real log_p_y_star;
    {
        int back_ptr[N, K];
        real best_logp[N, K];
        for (k in 1:K) {
            best_logp[1,k] = 0;
            for(x in 1:n) {
                for(y in x:n) {
                    best_logp[1,k] = best_logp[1,k] + poisson_log(interactions[1,x,y], exp(Ws[k][x,y]));
                }
            }
        }
    
        for (t in 2:N) {
            for (k in 1:K) {
                best_logp[t,k] = negative_infinity();
                for(j in 1:K) {
                    real logp;
                    logp = best_logp[t-1,j] + log(phi[j,k]);
                    for(x in 1:n) {
                        for(y in x:n) {
                            logp = logp + poisson_log(interactions[t,x,y], exp(Ws[k][x,y]));
                        }
                    }
                    if (logp > best_logp[t,k]) {
                        back_ptr[t,k] = j;
                        best_logp[t,j] = logp;
                    }

                }
            }
        }

        log_p_y_star = max(best_logp[N]);
        for(k in 1:K) {
            if(best_logp[N,k] == log_p_y_star) {
                y_star[N] = k;
            }
        }

        for(t in 1:(N-1)){
            y_star[N - t] = back_ptr[N - t + 1, y_star[N - t + 1]];
        }
    } 
}
"""

In [4]:
sm_infer = pystan.StanModel(model_code=function_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9a9acaf29f4c21ae6131b0ec502c255b NOW.


In [12]:
n = 12
p = 2
d = int(n * p - p*(p+1)/2)
K = 3
all_nodes_dat = all_nodes[6]

iters_infer = 3000
data_infer = {'n' : n, 'p' : p, 'd' : d, 'K': K, 'N' : all_nodes_dat.shape[0], 'interactions' : all_nodes_dat, 'beta' : [1/K for _ in range(K)]}
fit_infer = sm_infer.sampling(data = data_infer, iter = iters_infer, chains = 1)
#fit_vi = sm_infer.vb(data=data_infer)

In [13]:
angles = fit_infer.extract()['theta']
transprobs = fit_infer.extract()['phi']
y_star = fit_infer.extract()['y_star']

In [14]:
print(fit_infer)

Inference for Stan model: anon_model_9a9acaf29f4c21ae6131b0ec502c255b.
1 chains, each with iter=3000; warmup=1500; thin=1; 
post-warmup draws per chain=1500, total post-warmup draws=1500.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
phi[0,0]       0.86  9.5e-3   0.06   0.73   0.82   0.87   0.91   0.97     46   1.08
phi[1,0]       0.04  2.3e-3   0.02   0.01   0.03   0.04   0.05   0.08     52   1.02
phi[2,0]     2.9e-3  5.3e-4 4.6e-3 2.0e-7 8.6e-5 1.1e-3 3.7e-3   0.02     75   1.01
phi[0,1]       0.13  8.8e-3   0.06   0.03   0.08   0.12   0.17   0.26     49   1.09
phi[1,1]       0.95  2.5e-3   0.02   0.91   0.94   0.95   0.96   0.98     54    1.0
phi[2,1]     9.7e-3  1.9e-3 8.7e-3 5.7e-5 2.8e-3 7.6e-3   0.01   0.03     21   1.04
phi[0,2]       0.01  1.9e-3   0.02 5.6e-6 5.1e-4 3.5e-3   0.02   0.07    121    1.0
phi[1,2]       0.01  1.6e-3 9.7e-3 1.9e-4 3.1e-3 8.3e-3   0.01   0.04     35   1.04
phi[2,2]       0.99  1.2e-3 9.6e-3   0.96   0.98   0.99

In [36]:
hist1 = go.Histogram(x = y_star[:,244])
pyo.iplot([hist1])

In [32]:
hist1 = go.Scatter(x = list(range(angles.shape[0])), y=angles[:,2,17])
#hist1 = go.Histogram(x = angles[:,0,5])
pyo.iplot([hist1])

In [ ]:
hist1 = go.Histogram(x = transprobs[:,2,0])
hist2 = go.Histogram(x = transprobs[:,2,1])
hist3 = go.Histogram(x = transprobs[:,2,2])
pyo.iplot([hist1, hist2, hist3])